In [1]:
# !git clone https://github.com/srijandas07/video_convolution.git


In [2]:
cd video_convolution/scripts/

/media/joris/uca-msc-dsai/dl_computer_vision/practice/video_convolution/scripts


In [3]:
!pwd

/media/joris/uca-msc-dsai/dl_computer_vision/practice/video_convolution/scripts


In [4]:
import os
import sys

import numpy as np
import pandas as pd
from keras.applications.vgg16 import VGG16, decode_predictions, preprocess_input
from keras.models import Model
from keras.preprocessing import image
from tensorflow.keras.utils import load_img, img_to_array

# To_Do: load VGG model
model = VGG16(weights="imagenet", include_top=True)


2023-02-08 10:10:03.995337: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-08 10:10:04.456542: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


ModuleNotFoundError: No module named 'google.protobuf'

In [ ]:
model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
#To_Do: create a model using VGG16 architecture to perform feature extraction
model = Model(inputs=model.input, outputs=model.get_layer('fc2').output)
f_size = 4096

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
file_path = 'video_convolution/data/p1_arrangingobjects_subAction510175554/'                           
n_files = sorted(os.listdir(file_path))
f_p = os.path.basename(os.path.normpath(file_path))
batch_size = 50
k = 0

In [ ]:
def data_load_batch(k, batch_size):
  t_file = []
  for img_file in n_files[0 + k : batch_size + k]:
    img_path = file_path + "/" + img_file
    img = image.load_img(img_path, target_size=(224, 224))
    x = img_to_array(img)
    t_file.append(preprocess_input(x))
  t_file = np.asarray(t_file)
  return t_file


In [6]:
images = []
features_conv = []

for i in range(0, int(len(n_files)/batch_size)):
    images = data_load_batch(k, batch_size)
    #To_Do: extract features from the created model providing batch as input
    features_conv.append(model.predict(images,batch_size=batch_size))
    k = k + batch_size




NameError: name 'n_files' is not defined

In [ ]:
features_conv = np.asarray(features_conv)
features_conv.shape


(4, 50, 4096)

In [ ]:
features_conv = np.reshape(features_conv, [features_conv.shape[0]*features_conv.shape[1], f_size])
features_conv.shape

(200, 4096)

In [ ]:
images_remain = len(n_files) - k
features_conv5_remain = []
if images_remain > 0:
   images = data_load_batch(k, images_remain)
   features_conv5_remain.append(model.predict(images, batch_size=images_remain))
features_conv5_remain = np.asarray(features_conv5_remain)                           
features_conv5_remain = np.squeeze(features_conv5_remain)
features_conv = np.vstack([features_conv, features_conv5_remain])
features_conv.shape

(232, 4096)

In [ ]:
np.savetxt('./'+f_p+'.csv.gz', features_conv, delimiter=',')

In [ ]:
def max_min_conv(video):
    frame_features  = np.loadtxt(video, delimiter=',')
    max_features = np.amax(frame_features, axis=0)
    min_features = np.amin(frame_features, axis=0)
    final_t1 = np.hstack([max_features, min_features])
    return final_t1

In [ ]:
import glob
path = './'
for video in glob.glob(path+'*.csv.gz'):
    desc = []
    features = max_min_conv(os.path.join(path, video))
    desc = np.hstack([desc, features.ravel()])
    np.savetxt('/content/video_convolution/results/video_descriptors/'+os.path.basename(video)+'.txt', desc, delimiter=',')